In [110]:
import pandas as pd
import functions

df = pd.read_csv("../data/raw/ecommerce_sales.csv")

In [ ]:
# Remove all empty columns and rows (all NaN values)
df_cleaned = df.dropna(axis=1, how='all')
df_cleaned = df.dropna(how='all')

df_cleaned

In [ ]:
# Clean Columns
# print(df_cleaned.columns)

df_cleaned = df_cleaned.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'nace_r2', 'indic_is',
       'unit', 'OBS_FLAG', 'CONF_STATUS'])

df_cleaned.rename(columns={"geo": "country", "TIME_PERIOD": "year", "OBS_VALUE": "sales"}, inplace=True)

df_cleaned


In [113]:
# Shortens the size_emp values
size_map = {
    "From 1 to 9 persons employed": "A(1-9)",
    "From 10 to 49 persons employed": "B(10-49)",
    "From 50 to 249 persons employed": "C(50-249)",
    "250 persons employed or more": "D(>=250)",
    "10 persons employed or more": "E(>=10)"
}
df_cleaned["size_emp"] = df["size_emp"].apply(lambda x: size_map[x])

In [ ]:
# Filter only European Union
df_euro_all = df_cleaned[(df_cleaned['country'].str.contains("Euro area")) & (df_cleaned['size_emp'] == "E(>=10)")]
df_euro_all = df_euro_all.replace(to_replace='Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)', value="Euro Area")
df_euro_all

In [115]:
# Filter out "Euro area" and "European Union"
df_cleaned = df_cleaned[~df_cleaned['country'].str.contains("Euro")]

# df_cleaned["country"].unique() # Checks 

In [ ]:
df_cleaned = functions.region_mapping(df_cleaned)

In [117]:
# Creates two different dataframes
df_sales_cat = df_cleaned[(df_cleaned["size_emp"] != "E(>=10)") & (df_cleaned["size_emp"] != "A(1-9)")] # Dataframe with enterprises divided in four sizes categories
df_sales_all = df_cleaned[df_cleaned["size_emp"] == "E(>=10)"] # Dataframe with enterprises in one categoriy encompasing all 4 previous categories

In [ ]:
# Reset indexes
df_sales_cat.reset_index(drop=True, inplace=True)

df_sales_all.reset_index(drop=True, inplace=True)
df_sales_all.drop(columns=["size_emp"], inplace=True) # Delete size column as there is only one size in DF

df_sales_all.head(5)


In [119]:
# create a dummy function to send to the exploration notebook
def import_df_ecommerce():
    return (
        df_sales_cat,
        df_sales_all,
        df_euro_all
        )